In [ ]:
# %cd drive/MyDrive/dl-assigment-2/

# !unzip nature_12K.zip

In [ ]:
!pip install wandb

In [ ]:
import cv2
import glob
import random
import numpy as np
import torch
from pandas.core.common import flatten
torch.manual_seed(7)
torch.cuda.empty_cache()
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from PIL import Image
from torch import  nn,optim
import torch.nn.functional as F
from tqdm import tqdm 
import wandb

In [ ]:
!wandb login my_id

In [ ]:
# constants
IMG_MODE = 'RGB'
TRAIN_LABEL = 'train'
TEST_LABEL = 'test'

class iNaturalist(Dataset):
    def __init__(self, image_paths, class_to_idx, transform):
        self.all_images = image_paths
        self.current_transform = transform
        self.class_to_idx = class_to_idx
        
    def __len__(self):
        return len(self.all_images)

    def __getitem__(self, idx):
        image_filepath = self.all_images[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        y = self.class_to_idx[image_filepath.split('/')[-2]]
        
        X = Image.fromarray(np.uint8(image)).convert(IMG_MODE)
        X = Image.fromarray(image.astype('uint8'), IMG_MODE)
        X = self.current_transform(X)

        return X, y

def create_data(data_type, data_path,  data_aug, image_shape, b_size):
    classes = [image_path.split('/')[-1] for image_path in glob.glob(data_path + '/*')]

    all_images = [glob.glob(image_path + '/*') for image_path in glob.glob(data_path + '/*')]
    all_images = list(flatten(all_images))

    idx_to_class,class_to_idx = dict(),dict()
    for i, j in enumerate(classes):
        idx_to_class[i] = j
        class_to_idx[j] = i

    non_aug_tran = transforms.Compose([transforms.Resize((image_shape)),
                                transforms.ToTensor()
                                    ])
    if data_type == TEST_LABEL:
        test_image_paths=all_images
        test_dataset= iNaturalist(test_image_paths,class_to_idx,non_aug_tran)
        test_loader = DataLoader(test_dataset, batch_size=b_size, shuffle=True)

        return test_loader


    random.shuffle(all_images)

    tr_paths, v_paths = all_images[:int(0.8*len(all_images))], all_images[int(0.8*len(all_images)):] 

    tr_data,v_data = iNaturalist(tr_paths,class_to_idx,non_aug_tran),iNaturalist(v_paths,class_to_idx,non_aug_tran)

    if data_aug:
        augu_tran = transforms.Compose([transforms.Resize((image_shape)),
                transforms.RandomRotation(degrees=30),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomGrayscale(p=0.2),
                transforms.ToTensor(),
                            ])

        tr_data = iNaturalist(tr_paths,class_to_idx,augu_tran)
        v_data = iNaturalist(v_paths,class_to_idx,augu_tran)  

    t_loader,v_loader = DataLoader(tr_data, batch_size=b_size, shuffle=True),DataLoader(v_data, batch_size=b_size, shuffle=True)
    return t_loader,v_loader


In [ ]:
class ConvolutionBlocks(nn.Module):
    def __init__(self, activation, batch_norm, size_filters, filter_organization, number_filters,num_conv_layers):
        super().__init__()
        self.activationFn=activation
        self.num_filters=[number_filters]
        self.batch_norm=batch_norm
        for i in range(1,num_conv_layers):
            self.num_filters.append(int(self.num_filters[i-1]*filter_organization))
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=self.num_filters[0],kernel_size=size_filters[0],stride=(1, 1),padding=(1, 1),bias=False)
        self.conv2 = nn.Conv2d(in_channels=self.num_filters[0],out_channels=self.num_filters[1],kernel_size=size_filters[1],stride=(1, 1),padding=(1, 1),bias=False)
        self.conv3 = nn.Conv2d(in_channels=self.num_filters[1],out_channels=self.num_filters[2],kernel_size=size_filters[2],stride=(1, 1),padding=(1, 1),bias=False)
        self.conv4 = nn.Conv2d(in_channels=self.num_filters[2],out_channels=self.num_filters[3],kernel_size=size_filters[3],stride=(1, 1),padding=(1, 1),bias=False)
        self.conv5 = nn.Conv2d(in_channels=self.num_filters[3],out_channels=self.num_filters[4],kernel_size=size_filters[4],stride=(1, 1),padding=(1, 1),bias=False)
        self.pool  = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.batchnorm1 = nn.BatchNorm2d(self.num_filters[0])
        self.batchnorm2 = nn.BatchNorm2d(self.num_filters[1])
        self.batchnorm3 = nn.BatchNorm2d(self.num_filters[2])
        self.batchnorm4 = nn.BatchNorm2d(self.num_filters[3])
        self.batchnorm5 = nn.BatchNorm2d(self.num_filters[4])

    def forward(self, x):
        if self.batch_norm == False: 
            x=self.pool(self.activationFn(self.conv1(x)))
            x=self.pool(self.activationFn(self.conv2(x)))
            x=self.pool(self.activationFn(self.conv3(x)))
            x=self.pool(self.activationFn(self.conv4(x)))
            x=self.pool(self.activationFn(self.conv5(x)))
            return x
        else:
            x= self.pool(self.activationFn(self.batchnorm1(self.conv1(x))))
            x= self.pool(self.activationFn(self.batchnorm2(self.conv2(x))))
            x= self.pool(self.activationFn(self.batchnorm3(self.conv3(x))))
            x= self.pool(self.activationFn(self.batchnorm4(self.conv4(x))))
            x= self.pool(self.activationFn(self.batchnorm5(self.conv5(x))))
            return x

class Model(nn.Module):
    def __init__(self, image_shape,dropout , activation, batch_norm, size_filters, filter_organization, 
                  number_filters , neurons_in_dense_layer,num_conv_layers):
        super().__init__()
        activationFn = {
            "ReLU" : nn.ReLU(),
            "LeakyReLU" : nn.LeakyReLU(),
            "GELU" : nn.GELU(),
            "SiLU" : nn.SiLU(),
            "Mish" : nn.Mish(),
            "ELU" : nn.ELU()
        }
        self.activation = activationFn[activation]
        self.conv_blocks = ConvolutionBlocks(activation = self.activation,
                                             batch_norm= batch_norm,
                                             size_filters= size_filters,
                                             filter_organization= filter_organization,
                                             number_filters= number_filters,
                                             num_conv_layers= num_conv_layers)

        sz=self.conv_blocks(torch.zeros(*(image_shape))).data.shape
        self.fully_conn_layer_1   = nn.Linear(sz[1] * sz[2] * sz[3],neurons_in_dense_layer,bias=True)  
        self.output_layer= nn.Linear(neurons_in_dense_layer,10,bias=True)   
        self.dropout=nn.Dropout(p=dropout)
        self.num_conv_layers = num_conv_layers
    def forward(self, x):
        x = self.conv_blocks(x)
        x = self.dropout(self.activation(self.fully_conn_layer_1(x.reshape(x.shape[0],-1))))
        x = F.softmax(self.output_layer(x),dim=1)
        return x


In [ ]:
def convertIntoPercentage(x,n,digit=4):
    return round((x / n) * 100, digit)

def evaluate(device, loader, model):
    ''' Function to calculate accuracy to see performance of our model '''
        
    Y_cap_num,N_val = 0,0
    loss = 0

    model.eval()

    with torch.no_grad():
        for X, Y in tqdm(loader, total=len(loader)):
            X,Y = X.to(device=device),Y.to(device=device)

            Y_cap = model(X)
            loss += nn.CrossEntropyLoss()(Y_cap, Y).item()

            _, predictions = Y_cap.max(1)

            N_val = N_val + predictions.size(0)
            
            Y_cap_num = Y_cap_num +  (predictions == Y).sum().item()
           
    acc = convertIntoPercentage(Y_cap_num , N_val)
    loss = loss/N_val
    return acc, loss

def train():

    torch.cuda.empty_cache()
    image_shape = (1,3,224,224)
    test_data_path = '/kaggle/input/nature-12k/inaturalist_12K/val/'
    train_data_path = '/kaggle/input/nature-12k/inaturalist_12K/train/'
    config_defaults = dict({
        "epochs" : 10,
        "batch_size": 64,
        'activation': 'relu',
        'learning_rate':0.001,
        "dropout": 0.3,
        "batch_norm": True,
        "data_aug": True,
        'size_filters':[7,5,5,3,3],
        'filter_organization': 2,
        'number_filters': 16,
        "neurons_in_dense_layer": 512
    })

    wandb.init(project="dl-assignment-2", entity="cs23m007",config = config_defaults)
    args = wandb.config

    wandb.run.name = 'ep-'+str(args.epochs)+'-lr-'+str(args.learning_rate)+'-bs-'+str(args.batch_size)+'-act-'+str(args.activation)+'-drt-'+str(args.dropout) \
                      +'-bn-'+ str(args.batch_norm)+ '-da-'+str(args.data_aug)+'-filt_sizes-'+str(args.size_filters) \
                      + '-filt_org-'+str(args.filter_organization)+'-ini_filt'+str(args.number_filters)+'-n_d-'+str(args.neurons_in_dense_layer)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = Model(image_shape= image_shape,
                  dropout= args.dropout,
                  activation= args.activation,
                  batch_norm= args.batch_norm,
                  size_filters= args.size_filters,
                  filter_organization= args.filter_organization,
                  number_filters= args.number_filters,
                  neurons_in_dense_layer= args.neurons_in_dense_layer,
                  num_conv_layers= 5
                ).to(device)

    optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)

    for epoch in range(args.epochs):
        model.train()
        test_loader = create_data("test",test_data_path,args.data_aug, image_shape[2:], args.batch_size)
        train_loader, valid_loader = create_data("train",train_data_path,args.data_aug,image_shape[2:], args.batch_size)

        train_correct, train_loss = 0, 0
        total_samples = 0
        for batch_id,(data,label) in enumerate(tqdm(train_loader)):
          
            data = data.to(device=device)
            targets = label.to(device=device)

            scores = model(data)
            loss = nn.CrossEntropyLoss()(scores, targets)
            train_loss += loss.item()
            
            _, predictions = scores.max(1)
            train_correct += (predictions == targets).sum()
            total_samples +=  predictions.size(0)
            
            optimizer.zero_grad()
            loss.backward()

            optimizer.step()

        
        train_loss /= total_samples
        train_acc = round((train_correct / total_samples).item()  * 100, 4)
        
       
        
        val_acc, val_loss = evaluate(device, valid_loader, model)
        test_acc, test_loss = evaluate(device, test_loader, model)
        
        wandb.log(
          {'train_acc': train_acc, 'val_acc': val_acc, 'test_acc': test_acc, 'train_loss': train_loss, 'val_loss': val_loss, 'test_loss': test_loss}
        )

        print('\nEpoch ', epoch, 'train_acc', train_acc, 'val_acc', val_acc, 'test_acc', test_acc, 'train_loss', train_loss, 'val_loss', val_loss, 'test_loss', test_loss) 

In [ ]:
'''
config = {
    "epochs" : 10,
    "batch_size": 64,
    'activation': 'relu',
    'learning_rate':0.001,
    "dropout": 0.3,
    "batch_norm": True,
    "data_aug": True,
    'size_filters':[7,5,5,3,3],
    'filter_organization': 2,
    'number_filters': 16,
    "neurons_in_dense_layer": 512
}
class DotDict:
    def __init__(self, dictionary):
        self.__dict__.update(dictionary)

args = DotDict(config)
train(args)
'''

In [ ]:
# wandb.login()
sweep_config = {
    "name" : "Assignment2_Part_A_Q2",
    "method" : "bayes",
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    "parameters" : {
        'number_filters': {
            'values': [16, 32, 64, 128]
        },
        'activation': {
            'values': ['ReLU', 'LeakyReLU','GELU','SiLU','Mish','ELU']
        },
        'filter_organization': {
            'values': [1, 2, 0.5]
        },
        "data_aug": {
              "values": [True,False]
        },
        "batch_norm": {
              "values": [True,False]
        },
        "dropout": {
            "values": [0,0.1,0.2,0.3]
        },
        "batch_size": {
            "values": [32, 64, 128]
        },
        "epochs" : {
            "values" : [10, 15, 20 , 25 , 30]
        },
        'learning_rate':{
            "values": [0.001,0.0001,0.0003,0.0005]
        },
        'size_filters':{
            'values': [[7,5,5,3,3], [11,9,7,5,3]]
        },
        "neurons_in_dense_layer": {
            "values": [32, 64, 128, 256, 512, 1024]
        }        
    }
}

sweep_id = wandb.sweep(sweep_config, project="dl-assignment-2", entity="cs23m007")


In [ ]:
wandb.agent(sweep_id, train, count = 50)
wandb.finish()